# Cleaning data for visualization
While the data is not committed to the repository to save space, each of the data sets used in this notebook may be accessed at the following link, downloaded and stored locally in the `data` subdirectory of the `propublica_hotspots` directory such that you can run this notebook:
- [ProPublica: Toxic Air Pollution Hot Spots](https://www.propublica.org/datastore/dataset/toxic-air-pollution-hot-spots)
- [2020 DEC Redistricting Data Race Table, geographically filtered by Place](https://data.census.gov/cedsci/table?g=0100000US%241600000&y=2020&d=DEC%20Redistricting%20Data%20%28PL%2094-171%29&tid=DECENNIALPL2020.P1)
- [2010 DEC Redistricting Data Race Table, geographically filtered by Block](https://data.census.gov/cedsci/table?d=DEC%20Redistricting%20Data%20%28PL%2094-171%29)


In [1]:
import pandas as pd
import json
import numpy as np

## Loading pollution data

In [2]:
data = json.load(open("../data/toxmaps_files_2022-03-15b/toxmaps_files_for_data_store/hotspot_perimeters_for_data_store.geojson"))
data = data["features"]
pollution_hotspot_df = pd.DataFrame([d["properties"] for d in data])

# Normalizing with EPA acceptable risk, which is 1/10,000
pollution_hotspot_df["avg_ilcr"] = pollution_hotspot_df["avg_ilcr"] * 10000
pollution_hotspot_df["max_ilcr"] = pollution_hotspot_df["max_ilcr"] * 10000

pollution_hotspot_df


,id,place,state,avg_ilcr,max_ilcr,area,pop
0,24228,Corpus Christi,Texas,0.294000,3.403039,0.006039,57150
1,27627,Catoosa,Oklahoma,0.213759,0.472415,0.000526,68
2,38309,Chicago,Illinois,0.252534,0.252534,0.000071,388
3,27293,Attica,Indiana,0.280931,0.280931,0.000069,5
4,1180,Watertown,Wisconsin,0.313408,0.418059,0.000145,872
...,...,...,...,...,...,...,...
1356,38536,Monaca,Pennsylvania,0.183536,0.183536,0.000070,0
1357,8056,Prairie du Sac,Wisconsin,0.178760,0.178760,0.000073,280
1358,209,Sheboygan,Wisconsin,0.175148,0.175148,0.000073,210
1359,9940,Hooksett,New Hampshire,0.186700,0.186700,0.000072,36


## Loading census data

In [ ]:
name_keys = ["city", "CDP", "town", "municipality", "borough", "village", "corporation", "unified government", "urban county", "consolidated government", "consolidated government \(balance\)", "metropolitan government \(balance\)", "metro government \(balance\)", "\(balance\)"]

def extract_city_name(x):
    item_found = False
    i = 0

    while not item_found and i < len(name_keys):
        if name_keys[i] in x:
            item_found = True
        else:
            i = i + 1

    if item_found:
        return x.split(" " + name_keys[i])[0]
    else:
        return x

In [ ]:
raw_census_df = pd.read_csv("../data/DECENNIALPL2020.P1_2022-07-19T162239/DECENNIALPL2020.P1_data_with_overlays_2022-04-27T142004.csv", skiprows=1)

In [ ]:
census_df = pd.DataFrame(raw_census_df)
census_df.columns = census_df.columns.str.strip()

# Filtering out unneeded columns
cols = [col for col in census_df.columns if "!!Total:!!Population of one race:!!" in col or col in ["id", "Geographic Area Name", "!!Total:"]]
census_df = census_df.loc[:, cols]

# Renaming columns
census_df.columns = [s.replace("!!Total:!!Population of one race:!!", "") for s in census_df.columns]
census_df = census_df.rename(columns={"!!Total:": "Total"})

# Splitting geographic area name into city and state (seperated by comma)
location_name_split = census_df["Geographic Area Name"].str.split(",", expand=True)
census_df = census_df.drop("Geographic Area Name", axis=1)
census_df["City"] = location_name_split[0]
census_df["State"] = location_name_split[1].str.strip()

# Filling in the state of places where the county is also stored in the geographic area name
census_df.loc[~location_name_split[2].isna(), "State"] = location_name_split[2].str.strip()

# Filtering out Puerto Rico
census_df = census_df.loc[census_df["State"] != "Puerto Rico"]

# Extracting city names without place type indicator
census_df["City"] = census_df["City"].apply(lambda x: extract_city_name(x))

census_df

,id,Total,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,City,State
0,1600000US0100100,133,95,34,0,0,0,3,Abanda,Alabama
1,1600000US0100124,2358,1165,1039,5,15,0,51,Abbeville,Alabama
2,1600000US0100460,4366,1741,2313,31,10,3,123,Adamsville,Alabama
3,1600000US0100484,659,624,0,9,2,1,7,Addison,Alabama
4,1600000US0100676,225,19,199,0,2,0,0,Akron,Alabama
...,...,...,...,...,...,...,...,...,...,...
31612,1600000US5684852,118,105,1,0,2,0,0,Woods Landing-Jelm,Wyoming
31613,1600000US5684925,4773,3944,6,66,22,3,313,Worland,Wyoming
31614,1600000US5685015,1644,1447,0,18,0,0,86,Wright,Wyoming
31615,1600000US5686665,131,123,0,0,0,0,0,Yoder,Wyoming


## Merging data sets

In [ ]:
df = census_df.merge(pollution_hotspot_df, how="left", left_on=["City", "State"], right_on=["place", "state"])
df

,id_x,Total,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,City,State,id_y,place,state,avg_ilcr,max_ilcr,area,pop
0,1600000US0100100,133,95,34,0,0,0,3,Abanda,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1600000US0100124,2358,1165,1039,5,15,0,51,Abbeville,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1600000US0100460,4366,1741,2313,31,10,3,123,Adamsville,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1600000US0100484,659,624,0,9,2,1,7,Addison,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1600000US0100676,225,19,199,0,2,0,0,Akron,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31870,1600000US5684852,118,105,1,0,2,0,0,Woods Landing-Jelm,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31871,1600000US5684925,4773,3944,6,66,22,3,313,Worland,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31872,1600000US5685015,1644,1447,0,18,0,0,86,Wright,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31873,1600000US5686665,131,123,0,0,0,0,0,Yoder,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Block data retrieval

In [ ]:
# Using Hawaii as example/proof of concept
hi_blocks_raw = pd.read_csv("../data/HI/DECENNIALPL2010.P1_2022-07-22T131852/DECENNIALPL2010.P1_data_with_overlays_2022-07-22T131846.csv", skiprows=1)
hi_blocks_raw

c:\Users\Jason D'Amico\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,Total,Total!!Population of one race,Total!!Population of one race!!White alone,Total!!Population of one race!!Black or African American alone,Total!!Population of one race!!American Indian and Alaska Native alone,Total!!Population of one race!!Asian alone,Total!!Population of one race!!Native Hawaiian and Other Pacific Islander alone,Total!!Population of one race!!Some Other Race alone,Total!!Two or More Races,...,Total!!Two or More Races!!Population of five races,Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander,Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race,Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race,Total!!Two or More Races!!Population of five races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Total!!Two or More Races!!Population of five races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Total!!Two or More Races!!Population of five races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Total!!Two or More Races!!Population of six races,Total!!Two or More Races!!Population of six races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Geographic Area Name
0,1000000US150010201001000,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Block 1000, Block Group 1, Census Tract 201, H..."
1,1000000US150010201001001,123,95,62,0,1,27,4,1,28,...,0,0,0,0,0,0,0,0,0,"Block 1001, Block Group 1, Census Tract 201, H..."
2,1000000US150010201001002,16,12,9,0,0,1,2,0,4,...,0,0,0,0,0,0,0,0,0,"Block 1002, Block Group 1, Census Tract 201, H..."
3,1000000US150010201001003,11,9,4,0,0,4,0,1,2,...,0,0,0,0,0,0,0,0,0,"Block 1003, Block Group 1, Census Tract 201, H..."
4,1000000US150010201001004,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Block 1004, Block Group 1, Census Tract 201, H..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25011,1000000US150099912000002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Block 0002, Block Group 0, Census Tract 9912, ..."
25012,1000000US150099912000003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Block 0003, Block Group 0, Census Tract 9912, ..."
25013,1000000US150099912000004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Block 0004, Block Group 0, Census Tract 9912, ..."
25014,1000000US150099912000005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Block 0005, Block Group 0, Census Tract 9912, ..."


In [ ]:
hi_blocks = pd.DataFrame(hi_blocks_raw)

# Cleaning columns
hi_blocks["Total"] = hi_blocks["Total"].astype(str)
hi_blocks["Total"] = hi_blocks["Total"].str.split("(").str[0]
hi_blocks["Total"] = hi_blocks["Total"].astype(int)

# Filtering out unneeded columns
cols = [col for col in hi_blocks.columns if "Total!!Population of one race!!" in col or col in ["id", "Total"]]
hi_blocks = hi_blocks.loc[:, cols]

# Renaming columns
hi_blocks.columns = [s.replace("Total!!Population of one race!!", "") for s in hi_blocks.columns]
hi_blocks = hi_blocks.rename(columns={"!!Total:": "Total"})

# Removing rows with no residents
hi_blocks = hi_blocks.loc[hi_blocks["Total"] > 0]

# Formatting ID for use in crosswalk
hi_blocks["id"] = hi_blocks["id"].str.replace("1000000US", "")
hi_blocks["id"] = pd.to_numeric(hi_blocks["id"])

hi_blocks

,id,Total,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone
0,150010201001000,1,1,0,0,0,0,0
1,150010201001001,123,62,0,1,27,4,1
2,150010201001002,16,9,0,0,1,2,0
3,150010201001003,11,4,0,0,4,0,1
4,150010201001004,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
24917,150090320002118,4,4,0,0,0,0,0
24918,150090320002119,4,4,0,0,0,0,0
24919,150090320002120,2,2,0,0,0,0,0
24930,150090320002131,3,2,0,0,0,0,0


## Crosswalk retrieval
This is the file that the EPA uses to match census-designated blocks with its grid map coordinates used in the measuring of pollution levels.

In [ ]:
hi_x_walk_raw = pd.read_csv("../data/HI/CensusBlock2010_Hawaii_810m.csv")
hi_x_walk_raw

,GridID,X,Y,BlockID00,UR,PCT_B_C,PCT_C_B,PCT_CP_B
0,34,254,-203,150010212022293,Z,0.547465,0.028747,0.000000
1,34,254,-202,150010212022293,Z,0.451824,0.023725,0.000000
2,34,257,-203,150010212022359,Z,0.649448,0.145474,0.000000
3,34,257,-202,150010212022359,Z,0.350130,0.078428,0.000000
4,34,258,-202,150010212022359,Z,0.000463,0.000104,0.000000
...,...,...,...,...,...,...,...,...
109999,34,115,58,150090315023000,Z,0.242369,0.068603,0.087691
110000,34,135,48,150090308002023,Z,0.797387,0.097005,0.000000
110001,34,136,48,150090308002023,Z,0.192064,0.023365,0.000000
110002,34,136,49,150090308002023,Z,0.010631,0.001293,0.000000


In [ ]:
hi_x_walk = pd.DataFrame(hi_x_walk_raw)

# Getting number of total and white-identifying in each block
hi_x_walk = hi_x_walk.merge(hi_blocks.loc[:, ["id", "Total", "White alone"]], left_on="BlockID00", right_on="id")
hi_x_walk = hi_x_walk.drop(columns="id")
hi_x_walk = hi_x_walk.rename(columns={"White alone": "White in block", "Total": "Block total"})

# Normalizing based on percentage
hi_x_walk["Number of white from block"] = hi_x_walk["White in block"] * hi_x_walk["PCT_B_C"]
hi_x_walk["Total from block"] = hi_x_walk["Block total"] * hi_x_walk["PCT_B_C"]

hi_x_walk

,GridID,X,Y,BlockID00,UR,PCT_B_C,PCT_C_B,PCT_CP_B,Block total,White in block,Number of white from block,Total from block
0,34,243,-199,150010212022003,Z,0.000027,0.007577,1.000095,91,49,0.001323,0.002457
1,34,243,-198,150010212022003,Z,0.000180,0.050827,1.000095,91,49,0.008820,0.016380
2,34,243,-188,150010212022003,Z,0.000013,0.003720,1.000095,91,49,0.000637,0.001183
3,34,244,-201,150010212022003,Z,0.000551,0.155338,1.000095,91,49,0.026999,0.050141
4,34,244,-200,150010212022003,Z,0.001667,0.470405,1.000095,91,49,0.081683,0.151697
...,...,...,...,...,...,...,...,...,...,...,...,...
41435,34,157,23,150090303011087,Z,1.000000,0.019423,0.018935,3,1,1.000000,3.000000
41436,34,114,57,150090315023000,Z,0.301733,0.085406,0.184975,483,317,95.649361,145.737039
41437,34,114,58,150090315023000,Z,0.455723,0.128993,0.794152,483,317,144.464191,220.114209
41438,34,115,58,150090315023000,Z,0.242369,0.068603,0.087691,483,317,76.830973,117.064227
